Данильченко Вадим

Классификация текстов писателей

In [0]:
from google.colab import files
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
%matplotlib inline

from google.colab import drive
from tqdm import trange

from keras.preprocessing.text import Tokenizer
from keras import utils

Using TensorFlow backend.


In [0]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# загрузим данные из архива диска в папку texts
!unzip -q drive/'My Drive'/data/'Тексты писателей.zip' -d /content/texts

In [0]:
! ls texts

'(Булгаков) Обучающая_5 вместе.txt'
'(Булгаков) Тестовая_2 вместе.txt'
'(Клиффорд_Саймак) Обучающая_5 вместе.txt'
'(Клиффорд_Саймак) Тестовая_2 вместе.txt'
'(Макс Фрай) Обучающая_5 вместе.txt'
'(Макс Фрай) Тестовая_2 вместе.txt'
'(О. Генри) Обучающая_50 вместе.txt'
'(О. Генри) Тестовая_20 вместе.txt'
'(Рэй Брэдберри) Обучающая_22 вместе.txt'
'(Рэй Брэдберри) Тестовая_8 вместе.txt'
'(Стругацкие) Обучающая_5 вместе.txt'
'(Стругацкие) Тестовая_2 вместе.txt'


In [0]:
# функция для чтения файла с небольшим препроцессингом
def read_text(filename):
  f = open(filename, 'r')
  text = f.read()
  text = text.replace('\n', ' ')
  return text

labels = ["О. Генри", "Стругацкие", "Булгаков", "Саймак", "Фрай", "Брэдберри"]
ClASSES = len(labels)

In [0]:
# загрузим тексты в обучающую и тестовую выборку
train_data = []
test_data = []

for i in trange(len(labels)):
  for file in os.listdir('texts'):
    if labels[i] in file:
      if 'Обучающая' in file:
        train_data.append(read_text('texts/'+file))
        print(file, 'добавлен в Обучающую выборку')
      if 'Тестовая' in file:
        test_data.append(read_text('texts/'+file))
        print(file, 'добавлен в Тестовую выборку')
  print()

100%|██████████| 6/6 [00:00<00:00, 39.46it/s]

(О. Генри) Обучающая_50 вместе.txt добавлен в Обучающую выборку
(О. Генри) Тестовая_20 вместе.txt добавлен в Тестовую выборку

(Стругацкие) Обучающая_5 вместе.txt добавлен в Обучающую выборку
(Стругацкие) Тестовая_2 вместе.txt добавлен в Тестовую выборку

(Булгаков) Тестовая_2 вместе.txt добавлен в Тестовую выборку
(Булгаков) Обучающая_5 вместе.txt добавлен в Обучающую выборку

(Клиффорд_Саймак) Тестовая_2 вместе.txt добавлен в Тестовую выборку
(Клиффорд_Саймак) Обучающая_5 вместе.txt добавлен в Обучающую выборку

(Макс Фрай) Обучающая_5 вместе.txt добавлен в Обучающую выборку
(Макс Фрай) Тестовая_2 вместе.txt добавлен в Тестовую выборку

(Рэй Брэдберри) Обучающая_22 вместе.txt добавлен в Обучающую выборку
(Рэй Брэдберри) Тестовая_8 вместе.txt добавлен в Тестовую выборку



In [0]:
%%time
# обучим токенизатор на совместных обучающих и тестовых текстах
MAX_WORD_COUNT = 20000

tokenizer = Tokenizer(num_words=MAX_WORD_COUNT, filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0', lower=True, split=' ', oov_token='UNK', char_level=False)
tokenizer.fit_on_texts(train_data+test_data)
items = list(tokenizer.word_index.items())

CPU times: user 3.51 s, sys: 68.4 ms, total: 3.58 s
Wall time: 3.58 s


In [0]:
# преобразуем тексты в id (все слова, индексы которых будут больше MAX_WORD_COUNT, будут иметь индекс слова UNK)
train_ids = tokenizer.texts_to_sequences(train_data)
test_ids = tokenizer.texts_to_sequences(test_data)

In [0]:
# функция будет делать срезы некоторой длины slice_len с шагом step в последовательности sequence, которая будет ей подана на вход
def get_slices(sequence, slice_len, step):
  slices = []
  max_len = len(sequence)

  start = 0
  while (start+step<=max_len):
    slices.append(sequence[start:start+slice_len])
    start+=step
  
  return slices

In [0]:
# функция добавляет к входящей последовательности индексов текстов значения в целевую переменную
def create_dataset(sequences, slice_len, step):
  classes = len(sequences)
  seq_slices = [] 
  # запишем срезы индексов длиной slice_len для каждого текста
  for i in trange(len(sequences)):
    seq_slices.append(get_slices(sequences[i], slice_len, step))

  x = []
  y = []
  # запишем каждый срез текста в один массив и метку класса для него в другой
  for cls in trange(classes):
      text = seq_slices[cls]
      for j in trange(len(text)):
          x.append(text[j])
          y.append(utils.to_categorical(cls, classes))
  x = np.asarray(x)
  y = np.asarray(y)

  return (x, y)  

In [0]:
slice_len = 10000
step = 1000

x_train, y_train = create_dataset(train_ids, slice_len, step)
x_test, y_test = create_dataset(test_ids, slice_len, step)

print('train shapes:', x_train.shape, y_train.shape)
print('test shapes:', x_test.shape, y_test.shape)

100%|██████████| 160/160 [00:00<00:00, 37604.43it/s]

100%|██████████| 313/313 [00:00<00:00, 37783.26it/s]

100%|██████████| 261/261 [00:00<00:00, 40769.93it/s]

100%|██████████| 251/251 [00:00<00:00, 41061.29it/s]

100%|██████████| 568/568 [00:00<00:00, 41206.69it/s]

100%|██████████| 53/53 [00:00<00:00, 28235.50it/s]

100%|██████████| 108/108 [00:00<00:00, 17808.11it/s]

100%|██████████| 132/132 [00:00<00:00, 19848.29it/s]

100%|██████████| 50/50 [00:00<00:00, 22314.88it/s]

100%|██████████| 196/196 [00:00<00:00, 36386.65it/s]

100%|██████████| 6/6 [00:00<00:00, 94.44it/s]


train shapes: (1767,) (1767, 6)
test shapes: (671,) (671, 6)


In [0]:
print(x_train[0])

[5801, 7, 2866, 2433, 1, 3, 1, 1, 11, 1, 5126, 3, 3163, 1, 1, 249, 882, 15, 264, 1664, 7, 1, 1, 534, 9, 2950, 1, 1, 3, 1600, 8492, 11768, 1050, 11, 29, 52, 1, 130, 119, 1, 123, 1467, 3426, 1, 900, 1, 1150, 16434, 1, 37, 188, 1, 65, 1, 1759, 3850, 1, 956, 1, 2265, 1, 2817, 3, 1, 10711, 2, 1, 17614, 1, 223, 1, 1, 11, 15386, 1, 139, 745, 1, 1, 1, 522, 249, 1, 2, 1, 1710, 1, 6269, 18, 1, 1, 387, 10267, 77, 16, 1566, 1, 10712, 14531, 1665, 7, 1, 1, 1, 1, 10712, 1, 3712, 10268, 51, 61, 1, 11769, 3654, 236, 249, 4544, 82, 1906, 8493, 4, 704, 3325, 3, 2433, 4458, 4206, 8, 7925, 14532, 1, 1311, 6625, 2266, 1, 913, 266, 1, 9139, 343, 7226, 223, 11770, 18, 1, 146, 8493, 826, 8, 6270, 9140, 5254, 4305, 31, 4545, 1, 2, 1, 1277, 1, 14533, 3528, 7, 8494, 2, 15387, 1, 2, 2867, 1, 1, 1, 14531, 1665, 1, 247, 883, 12330, 1, 1, 1, 134, 43, 1, 1, 6271, 2, 3851, 6, 1665, 7, 888, 1, 8, 1, 780, 13711, 2, 4993, 3, 186, 1, 1, 4377, 1, 2123, 6, 883, 1665, 336, 12, 3921, 921, 7926, 14534, 91, 3, 1246, 17615, 40, 

In [0]:
# приведем к формату bag of words по длине словаря MAX_WORD_COUNT
x_train_bow = tokenizer.sequences_to_matrix(x_train.tolist())
x_test_bow = tokenizer.sequences_to_matrix(x_test.tolist())
x_train_bow.shape

(1767, 20000)

In [0]:
# создадим граф вычислений нейросети
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU
from keras.optimizers import Adam
from keras.regularizers import l1

model = Sequential()
model.add(Dense(500, activation='linear', input_dim=MAX_WORD_COUNT, name='input'))
model.add(Dropout(0.2, name='dropout_1'))
model.add(Dense(300, name='dense_2'))
model.add(LeakyReLU(alpha=0.2))
model.add(BatchNormalization(name='batch_norm'))
model.add(Dropout(0.2, name='dropout_2'))
model.add(Dense(200, activation='relu', name='dense_3'))
model.add(Dropout(0.2, name='dropout_3'))
model.add(Dense(100, activation='softmax', name='dense_4'))
model.add(Dropout(0.1, name='dropout_4'))
model.add(Dense(ClASSES, activation='softmax', name='output'))
model.compile(loss='categorical_crossentropy',
              metrics=['accuracy'],
              optimizer=Adam(learning_rate=1e-4))

model.fit(x_train_bow,
          y_train,
          validation_data=(x_test_bow, y_test),
          epochs=20,
          batch_size=256,
          verbose=1)

Train on 1767 samples, validate on 671 samples
Epoch 1/20
1767/1767 [==============================] - 1s 451us/step - loss: 1.7629 - accuracy: 0.5461 - val_loss: 1.7823 - val_accuracy: 0.4098
Epoch 2/20
1767/1767 [==============================] - 0s 241us/step - loss: 1.6853 - accuracy: 0.9276 - val_loss: 1.7576 - val_accuracy: 0.5723
Epoch 3/20
1767/1767 [==============================] - 0s 243us/step - loss: 1.6427 - accuracy: 0.9287 - val_loss: 1.7385 - val_accuracy: 0.6513
Epoch 4/20
1767/1767 [==============================] - 0s 235us/step - loss: 1.6211 - accuracy: 0.8942 - val_loss: 1.7266 - val_accuracy: 0.7303
Epoch 5/20
1767/1767 [==============================] - 0s 246us/step - loss: 1.6107 - accuracy: 0.8930 - val_loss: 1.7171 - val_accuracy: 0.8465
Epoch 6/20
1767/1767 [==============================] - 0s 240us/step - loss: 1.6037 - accuracy: 0.9179 - val_loss: 1.7083 - val_accuracy: 0.8733
Epoch 7/20
1767/1767 [==============================] - 0s 244us/step - loss: